<a href="https://colab.research.google.com/github/JJEEEFFFF/CE889-Labs/blob/main/Assignmet/EEG_Assignment_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, AveragePooling2D, Conv2D
from sklearn.model_selection import cross_val_score, cross_validate, KFold, StratifiedKFold, cross_val_predict
from keras import layers, models
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
import numpy as np
import scipy.io as io
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.model_selection import train_test_split

# **Load and Prepare the data**

In [2]:
data = io.loadmat('/content/drive/MyDrive/Neural/WLDataCW.mat')
print('Length: ' + str(len(data)))
print(data.keys())
print(data['__globals__'])
data_only = data["data"]
label = data["label"]
print(data_only.shape)
print(label.shape)

X = data['data']
# y = mat_contents['label']
data['label'] = data['label'].reshape(360)

y_convert = pd.get_dummies(data['label'])
# X = X.reshape(62*512,360)
X = X.reshape(360, 62*512)

y_convert = np.array(y_convert)
# y = y_convert.reshape(2,360)
y = y_convert

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Length: 5
dict_keys(['__header__', '__version__', '__globals__', 'data', 'label'])
[]
(62, 512, 360)
(1, 360)


In [3]:
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train)
y_new = np.argmax(y, axis=1)
kfold = KFold(n_splits=5, shuffle=True)
accuracies = cross_val_score(estimator=neigh,
                             X=X,
                             y=y_new,
                             cv=kfold, scoring='accuracy')
print(accuracies.mean())

0.43611111111111106


In [4]:
def make_classifier():
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=31744))
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dropout(rate=0.2))
    model.add(Flatten())
    model.add(layers.Dense(2, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['accuracy'])
    return model


In [6]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
classifier = KerasClassifier(build_fn=make_classifier, verbose=1)

# Fit data to model
classifier.fit(X_train, y_train, batch_size=64, epochs=50)

y_new = np.argmax(y, axis=1)

accuracies = cross_val_score(estimator=classifier,
                             X=X,
                             y=y_new,
                             cv=kfold, scoring='accuracy')
print(accuracies.mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/50
5/5 [==============================] - 2s 41ms/step - loss: 0.4872 - accuracy: 0.4896
Epoch 2/50
5/5 [==============================] - 0s 41ms/step - loss: 0.5149 - accuracy: 0.4618
Epoch 3/50
5/5 [==============================] - 0s 44ms/step - loss: 0.4479 - accuracy: 0.5451
Epoch 4/50
5/5 [==============================] - 0s 40ms/step - loss: 0.4540 - accuracy: 0.5451
Epoch 5/50
5/5 [==============================] - 0s 44ms/step - loss: 0.4462 - accuracy: 0.5521
Epoch 6/50
5/5 [==============================] - 0s 38ms/step - loss: 0.4983 - accuracy: 0.5000
Epoch 7/50
5/5 [==============================] - 0s 42ms/step - loss: 0.4933 - accuracy: 0.5000
Epoch 8/50
5/5 [==============================] - 0s 43ms/step - loss: 0.5012 - accuracy: 0.4931
Epoch 9/50
5/5 [==============================] - 0s 38ms/step - loss: 0.4550 - accuracy: 0.5382
Epoch 10/50
5/5 [==============================] - 0s 39ms/step - loss: 0.4549 - accuracy: 0.5417
Epoch 11/50
5/5 [============